# 15 Node Triangular Peg Puzzle

## Coordinate system
Rather than do something boring like label the nodes 1-15, I divide the nodes into 3 symmetric groups: A, B, and C.
## Field and Particles
I use "field" to capture the geometric constraints for moving the "particles".
Additional rules for particle interactions are described in: test_possible_move, and execute_move.
### Notes about the field
Most nodes have two possible moves associated with them, but the midpoints of the edges of the triangles each have 4.
### Notes about the particles
The initial configuration of the particles (particles_15) breaks radial symmetry.
Fun fact:  we know the positions of the particles absolutely and their velocities are irrelevant.

In [1]:
field_15 = {'A1': [('A3', 'A2'), ('C3', 'A5')],
    'A2': [('B5', 'A3'), ('C4', 'A4')],
    'A3': [('A1', 'A2'), ('B1', 'B5'),('B3', 'B4'), ('C3', 'A4')],
    'A4': [('B2', 'B4'), ('C5', 'C4')],
    'A5': [('B4', 'A4'), ('C2', 'C3')],
    'B1': [('B3', 'B2'), ('A3', 'B5')],
    'B2': [('C5', 'B3'), ('A4', 'B4')],
    'B3': [('B1', 'B2'), ('C1', 'C5'),('C3', 'C4'), ('A3', 'B4')],
    'B4': [('C2', 'C4'), ('A5', 'A4')],
    'B5': [('C4', 'B4'), ('A2', 'A3')],
    'C1': [('C3', 'C2'), ('B3', 'C5')],
    'C2': [('A5', 'C3'), ('B4', 'C4')],
    'C3': [('C1', 'C2'), ('A1', 'A5'),('A3', 'A4'), ('B3', 'C4')],
    'C4': [('A2', 'A4'), ('B5', 'B4')],
    'C5': [('A4', 'C4'), ('B2', 'B3')]}

particles_15 = {'A1': 'X',
    'A2': 'X',
    'A3': 'X',
    'A4': 'O',
    'A5': 'X',
    'B1': 'X',
    'B2': 'X',
    'B3': 'X',
    'B4': 'X',
    'B5': 'X',
    'C1': 'X',
    'C2': 'X',
    'C3': 'X',
    'C4': 'X',
    'C5': 'X'}

game_tree_15 = {}
todo_list_15 = []
game_status_15 = {}

* todo_list collects the unexamined game nodes.  The values are a sequence of moves, represented by a tuple of tuples.
* game_tree collects the generated possible next moves.  The keys are the jump paths, represented by tuples of tuples.
The values include the available next moves, and the current state of the particles.
* game_status collects the "Terminal" versus "Non-terminal" status of each node in game_tree.

In [2]:
def test_possible_move(particles, start_pos, middle_pos, end_pos):
    if particles[start_pos] == 'X' and particles[middle_pos] == 'X' and particles[end_pos] =='O'  :
        return(True)
    else:
        return(False)

In [3]:
def execute_move(particles, start_pos, middle_pos, end_pos):
    particles[start_pos] = 'O'
    particles[middle_pos] = 'O'
    particles[end_pos] = 'X'

In [4]:
def find_possible_moves(field, particles):
    possible_moves_list = []
    for start_pos, end_list in field.items():
        for (end_pos, middle_pos) in end_list:
            if test_possible_move(particles, start_pos, middle_pos, end_pos):
                possible_moves_list.append((start_pos, middle_pos, end_pos))
    return possible_moves_list

In [5]:
def generate_all_games(field, particles, game_tree, todo_list, game_status):
    game_tree[((None,),)] = [find_possible_moves(field, particles), particles]
    todo_list.append(((None,),))
    while len(todo_list) > 0 and len(game_status) < 2000000:
        game_node = todo_list.pop()
        branches = game_tree[game_node][0]
        particles_before = game_tree[game_node][1]
        for branch in branches:
            start_pos, middle_pos, end_pos = branch
            particles_after = particles_before.copy()
            execute_move(particles_after, start_pos, middle_pos, end_pos)
            jump_list = list(game_node)
            jump_list.append((start_pos, end_pos))
            new_game_node = tuple(jump_list)
            game_tree[new_game_node] = [find_possible_moves(field, particles_after), particles_after]
            todo_list.append(new_game_node)
        if len(branches) == 0:
            game_status[game_node] = 'Terminal'
        else:
            game_status[game_node] = 'Non-terminal'

In [6]:
def analyze_games_by_length(game_status):
    victory_count = 0
    game_count = 0
    distinct_final_moves = {}
    distinct_start_final_moves = {}
    for jump_tuples, end_status in game_status.items():
        if end_status == 'Terminal':
            game_count +=1 
        if end_status == 'Terminal' and len(jump_tuples) > 13:
            victory_count += 1
            distinct_final_moves[jump_tuples[-1]] = 1
            distinct_start_final_moves[(jump_tuples[1], jump_tuples[-1])] = 1
            print('Game length: ' + str(len(jump_tuples) - 1) + '\n')
            print('Moves: ' + str(jump_tuples) +'\n')
    print("Total paths to victory: " + str(victory_count))
    print("Total paths: " + str(game_count))
    distinct_final_moves_list = list(distinct_final_moves.keys())
    print("Distinct final moves: " + str(distinct_final_moves_list))
    distinct_start_final_moves_list = list(distinct_start_final_moves.keys())
    print("Distinct start, final move pairs: " + str(distinct_start_final_moves_list))
    

In [7]:
generate_all_games(field_15, particles_15, game_tree_15, todo_list_15, game_status_15)
analyze_games_by_length(game_status_15)


Game length: 13

Moves: ((None,), ('C5', 'A4'), ('B5', 'C4'), ('C2', 'B4'), ('B2', 'C5'), ('A5', 'C2'), ('C1', 'C3'), ('A2', 'B5'), ('C3', 'A3'), ('B5', 'A2'), ('A1', 'A3'), ('A3', 'B3'), ('C5', 'B2'), ('B1', 'B3'))

Game length: 13

Moves: ((None,), ('C5', 'A4'), ('B5', 'C4'), ('C2', 'B4'), ('B2', 'C5'), ('A5', 'C2'), ('A2', 'B5'), ('C1', 'C3'), ('C3', 'A3'), ('B5', 'A2'), ('A1', 'A3'), ('A3', 'B3'), ('C5', 'B2'), ('B1', 'B3'))

Game length: 13

Moves: ((None,), ('C5', 'A4'), ('B5', 'C4'), ('C2', 'B4'), ('B2', 'C5'), ('A2', 'B5'), ('A5', 'C2'), ('C1', 'C3'), ('C3', 'A3'), ('B5', 'A2'), ('A1', 'A3'), ('A3', 'B3'), ('C5', 'B2'), ('B1', 'B3'))

Game length: 13

Moves: ((None,), ('C5', 'A4'), ('B5', 'C4'), ('C2', 'B4'), ('A5', 'C2'), ('C1', 'C3'), ('B2', 'C5'), ('A2', 'B5'), ('C3', 'A3'), ('B5', 'A2'), ('A1', 'A3'), ('A3', 'B3'), ('C5', 'B2'), ('B1', 'B3'))

Game length: 13

Moves: ((None,), ('C5', 'A4'), ('B5', 'C4'), ('C2', 'B4'), ('A5', 'C2'), ('C1', 'C3'), ('A2', 'B5'), ('C3', 'A3'), 


Game length: 13

Moves: ((None,), ('B2', 'A4'), ('C2', 'B4'), ('A5', 'C2'), ('A2', 'C4'), ('C1', 'C3'), ('B5', 'A2'), ('A1', 'A3'), ('C5', 'B2'), ('A3', 'B3'), ('B2', 'C5'), ('C3', 'B3'), ('C5', 'B2'), ('B1', 'B3'))

Game length: 13

Moves: ((None,), ('B2', 'A4'), ('C2', 'B4'), ('A5', 'C2'), ('A2', 'C4'), ('B5', 'A2'), ('C5', 'B2'), ('C1', 'C3'), ('C3', 'B3'), ('B2', 'C5'), ('A1', 'A3'), ('A3', 'B3'), ('C5', 'B2'), ('B1', 'B3'))

Game length: 13

Moves: ((None,), ('B2', 'A4'), ('C2', 'B4'), ('A5', 'C2'), ('A2', 'C4'), ('B5', 'A2'), ('C5', 'B2'), ('C1', 'C3'), ('C3', 'B3'), ('A1', 'A3'), ('B2', 'C5'), ('A3', 'B3'), ('C5', 'B2'), ('B1', 'B3'))

Game length: 13

Moves: ((None,), ('B2', 'A4'), ('C2', 'B4'), ('A5', 'C2'), ('A2', 'C4'), ('B5', 'A2'), ('C5', 'B2'), ('C1', 'C3'), ('A1', 'A3'), ('C3', 'B3'), ('B2', 'C5'), ('A3', 'B3'), ('C5', 'B2'), ('B1', 'B3'))

Game length: 13

Moves: ((None,), ('B2', 'A4'), ('C2', 'B4'), ('A5', 'C2'), ('A2', 'C4'), ('B5', 'A2'), ('C5', 'B2'), ('C1', 'C3'),